In [1]:
import face_recognition
from face_recognition import face_locations
import os
import pandas as pd
import torch
import h5py
import scipy.io
import numpy as np
from tqdm import tqdm

In [2]:
img_dir = '/mnt/hdd1/data/face/LFWA/cropped/original/'

In [3]:
label_mat = '/mnt/hdd1/data/face/LFWA/label.mat'
label = scipy.io.loadmat(label_mat)['label']
name_mat = '/mnt/hdd1/data/face/LFWA/name.mat'
name = scipy.io.loadmat(name_mat)['name']
name = [s[0].split('\\')[1] for s in name.tolist()[0]]
attr_name_mat = '/mnt/hdd1/data/face/LFWA/attrname.mat'
attr_name = scipy.io.loadmat(attr_name_mat)['AttrName']
attr_name = [str(s[0]) for s in attr_name.tolist()[0]]

In [4]:
df_label = pd.DataFrame(label, columns=attr_name, index=name)

In [5]:
len(df_label)

13143

In [6]:
vecs = []
fnames = []
i = 0
for fname in tqdm(df_label.index):
    i += 1
    img_path = os.path.join(img_dir, fname)
    X_img = face_recognition.load_image_file(img_path)
    X_faces_loc = face_locations(X_img)
    if len(X_faces_loc) != 1:
        continue
    faces_encoding = face_recognition.face_encodings(X_img, known_face_locations=X_faces_loc)[0]
    
    vecs.append(faces_encoding)
    fnames.append(fname)
    
df_feat = pd.DataFrame(vecs, index=fnames)
df_label = df_label[df_label.index.isin(df_feat.index)]
df_feat.sort_index(inplace=True)
df_label.sort_index(inplace=True)

df_feat.to_csv('/mnt/hdd1/data/face/LFWA/feature.csv')
df_label.to_csv('/mnt/hdd1/data/face/LFWA/label.csv')

100%|██████████| 13143/13143 [09:54<00:00, 22.12it/s]
